In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
from utils import *

In [2]:
train = pd.read_csv('data/train_ksmod.csv').fillna(0)
test = pd.read_csv('data/test_ksmod.csv').fillna(0)
sample = pd.read_csv('data/sample_submission.csv')

In [3]:
sess = tf.InteractiveSession()

In [4]:
batch_size = 10

In [5]:
x = tf.placeholder('float', shape=[None, train.iloc[:, 3:].shape[1]])
y_ = tf.placeholder('float', shape=[None, 3])
l2 = tf.placeholder('float')
hidden1 = 2000
hidden2 = 100

with tf.name_scope('hidden1'):
    weights = tf.Variable(
        tf.truncated_normal([train.iloc[:, 3:].shape[1], hidden1],
                            stddev=1.0 / np.sqrt(float(train.iloc[:, 3:].shape[1]))),
        name='weights')
    biases = tf.Variable(tf.zeros([hidden1]),
                         name='biases')
    hidden1_tensor = tf.nn.relu(tf.matmul(x, weights) + biases)
    reg_hidden1 = tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases)

with tf.name_scope('hidden2'):
    weights = tf.Variable(
        tf.truncated_normal([hidden1, hidden2],
                            stddev=1.0 / np.sqrt(float(hidden1))),
        name='weights')
    biases = tf.Variable(tf.zeros([hidden2]),
                         name='biases')
    hidden2_tensor = tf.nn.relu(tf.matmul(hidden1_tensor, weights) + biases)
    reg_hidden2 = tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases)

with tf.name_scope('softmax_linear'):
    weights = tf.Variable(
        tf.truncated_normal([hidden2, 3],
                            stddev=1.0 / np.sqrt(float(hidden2))),
        name='weights')
    biases = tf.Variable(tf.zeros([3]),
                         name='biases')
    logits = tf.matmul(hidden2_tensor, weights) + biases
    reg_hidden3 = tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases)

sess.run(tf.initialize_all_variables()) 
y = tf.nn.softmax(logits)
cross_entropy = - tf.reduce_mean(y_ * tf.log(y) + 1e-15) + l2 * (reg_hidden1 + reg_hidden2 + reg_hidden3)
train_step = tf.train.GradientDescentOptimizer(.001).minimize(cross_entropy)
train_x, valid_x, train_y, valid_y = train_test_split(
        train.iloc[:, 3:],
        train.loc[:, 'fault_severity'],
        test_size=0.3333)
labels = tf.expand_dims(np.asarray(train.loc[:, 'fault_severity'].values, dtype='int32'), 1)
indices = tf.expand_dims(tf.range(0, train.shape[0]), 1)
concated = tf.concat(1, [indices, labels])
train_onehot_labels = tf.sparse_to_dense(
    concated, tf.pack([train.shape[0], sample.shape[1]-1]), 1.0, 0.0).eval()
for i in range(train_x.shape[0]//batch_size):
    train_step.run(
        feed_dict={
            x: train.iloc[batch_size*i:batch_size*(i+1), 3:].values,
            y_: train_onehot_labels[batch_size*i:batch_size*(i+1), :],
            l2: 0.1})

In [6]:
prob = y.eval(feed_dict={x: test.iloc[:, 2:]})

In [7]:
nn = pd.DataFrame({
    'id': sample.loc[:, 'id'].values, 
    'predict_0': prob[:, 0],
    'predict_1': prob[:, 1],
    'predict_2': prob[:, 2]})

In [8]:
nn.to_csv('tnn.csv', index=False)